In [12]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [13]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

mtcaic3_path = kagglehub.competition_download('mtcaic3')

print('Data source import complete.')


100%|██████████| 533M/533M [00:07<00:00, 71.5MB/s]

Extracting files...


Data source import complete.


In [99]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from sklearn.preprocessing import LabelEncoder

In [100]:
DATA = mtcaic3_path
columns = ['FZ', 'C3', 'CZ', 'C4', 'PZ', 'PO7', 'OZ', 'PO8', 'AccX',
       'AccY', 'AccZ', 'Gyro1', 'Gyro2', 'Gyro3']
le = LabelEncoder()

In [105]:
class EEGData(Dataset):
    def __init__(self,path,task:str,train:bool=True):
        """
            path: string
            type: Literal["train","test"]
        """

        self.dataframe = pd.read_csv(path)
        self.dataframe = self.dataframe[self.dataframe['task'] == task]
        self.dataframe['label'] = le.fit_transform(self.dataframe['label'])
        self.train = train

    def __getitem__(self,index):
        if self.train:
            path = 'train/'
        else:
            path = 'test/'
        datapoint = self.dataframe.iloc[index]

        eeg = pd.read_csv(f'{DATA}/{datapoint.task}/{path}/{datapoint.subject_id}/{datapoint.trial_session}/EEGdata.csv')
        mag = len(eeg)
        trial  = int(datapoint['trial'])
        lower = mag*(trial-1)//10
        upper = mag*trial//10
        eeg = eeg[lower:upper]
        eeg = eeg[columns]
        eeg = torch.from_numpy(np.array(eeg,dtype='float32')).permute(1,0)
        if self.train:
            return eeg,datapoint['label']
        else:
            return eeg

    def __len__(self):
        return len(self.dataframe)

In [106]:
train_mi = EEGData(f'{DATA}/train.csv','MI',True)

In [84]:
train_mi[0][0].shape


torch.Size([14, 2250])

In [107]:
train_mi_loader = DataLoader(train_mi,batch_size=10,shuffle=True)

In [92]:
model = torch.nn.Sequential(
    nn.Conv1d(in_channels=len(columns), out_channels=len(columns)//2, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.Conv1d(len(columns)//2,out_channels=len(columns)//4,kernel_size=3,padding=1),
    nn.ReLU(),
    nn.Conv1d(len(columns)//4,out_channels=len(columns)//8,kernel_size=3,padding=1),
    nn.ReLU(),
    nn.Conv1d(len(columns)//8,out_channels=1,kernel_size=3,padding=1),
    nn.ReLU(),
    nn.Flatten(),
    nn.Linear(2250,1),
    nn.Sigmoid()
)

In [108]:
for X,y in train_mi_loader:
    print(y)
    break

tensor([0, 1, 1, 0, 1, 1, 1, 1, 1, 0])


In [ ]:
# prompt: train the model for a binary classification

# Training loop
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 1
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for i, (inputs, labels) in enumerate(train_mi_loader):
        optimizer.zero_grad()

        outputs = model(inputs)
        # Ensure labels are floats for BCELoss
        labels = labels.float().unsqueeze(1)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        predicted = (outputs > 0.5).squeeze().long()
        correct_predictions += (predicted == labels.squeeze().long()).sum().item()
        total_predictions += labels.size(0)


    epoch_loss = running_loss / len(train_mi_loader)
    epoch_accuracy = correct_predictions / total_predictions
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}')

print('Finished Training')